<a href="https://colab.research.google.com/github/dodonas/CC_Ext_2/blob/main/final/CLOUD_TECH_EXT_2_CONSUMER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Afeka - 65215 - Ext 2 - Kafka and Spark Streaming

# Submitter: Andrey Dodon - 317858298

# Setup

Install the required kafka package and get AFINN-111.txt

In [1]:
%%capture
!pip install confluent_kafka

!wget https://github.com/dodonas/CC_Ext_2/blob/main/AFINN-111.txt

In [20]:
from confluent_kafka import Consumer, KafkaError, KafkaException
from time import time, sleep
from pathlib import Path
import sys
import os

In [3]:
def twoSecondsPass(startTime):
  return (time() - startTime) > 2

In [38]:
def writeToRandJ(suffix, batch):
  os.path.join('Outputs', 'RandJ.part' + str(suffix))
  fileName = Path('RandJ.part' + str(suffix))
  fileName.touch(exist_ok=True)
  with open(fileName, 'w+') as f:
    for item in batch:
      f.write("%s\n" % item)
  sys.stderr.write('File [%s] has been created\n' % (fileName))
  suffix += 1
  return [], suffix

In [ ]:
running = True

def basic_consume_loop(consumer, topics):
    try:        
        consumer.subscribe(topics)        
        startTime = time()
        suffix = 1
        batch = []      
        while running:         
            msg = consumer.poll(timeout=2.0) 
            if twoSecondsPass(startTime):                               
                if len(batch) > 0:
                    batch, suffix = writeToRandJ(suffix, batch)               
                startTime = time()                
            if msg is None: continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                     (msg.topic(), msg.partition(), msg.offset()))
                elif msg.error():
                    raise KafkaException(msg.error())
            else:
                batch.append(msg.value().decode('utf-8'))
                
    finally:
        consumer.close()

def shutdown():
    running = False

In [ ]:
conf = {'bootstrap.servers': "localhost:9092",
        'group.id': "afeka-group",
        'auto.offset.reset': 'smallest'}
consumer = Consumer(conf)

basic_consume_loop(consumer, ['RomeoAndJuliet'])